In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
#hide
from fastbook import *
from IPython.display import display,HTML

In [2]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [3]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [4]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [5]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.998896,3.901318,0.300748,49.467583,20:15


In [6]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.768958,3.760451,0.316702,42.967781,21:11
1,3.710859,3.710861,0.322022,40.889004,21:26
2,3.608949,3.655514,0.328542,38.687382,21:40
3,3.567884,3.622703,0.332413,37.438622,21:38
4,3.490791,3.600577,0.335219,36.619354,21:57
5,3.439936,3.580077,0.337952,35.876301,22:07
6,3.352408,3.573172,0.339503,35.629425,22:25
7,3.311336,3.569648,0.340494,35.504078,22:31
8,3.245657,3.573910,0.340680,35.655750,22:47
9,3.205735,3.578315,0.340503,35.813129,23:00


In [7]:
learn.save_encoder('finetuned')

In [8]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [9]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [10]:
learn = learn.load_encoder('finetuned')

In [11]:

learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.244611,0.181000,0.930920,00:27


In [12]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.222388,0.167875,0.936960,00:30


In [13]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.191986,0.148612,0.945280,00:40


In [14]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.156988,0.148744,0.946360,00:50
1,0.143647,0.149443,0.947200,00:51


In [15]:
t=learn.export()


In [17]:
!./gdrive upload {t}

/bin/bash: ./gdrive: No such file or directory


In [18]:
t